In [5]:
%matplotlib inline
%run 'bin/notebook_utils.py'

In [6]:
exp1_sessions = pd.read_csv('../data/exp1/sessions.csv', index_col=0)
exp1_pilots = pd.read_csv('../data/exp1/pilots.csv', index_col=0)
exp1_units = pd.read_csv('../data/exp1/units.csv', index_col=0)

In [7]:
print 'Experiment 1: Baseline: Total number of successful runs: %s' % exp1_sessions.shape[0]
print 'Experiment 1: Baseline: Total number of pilots: %s' % exp1_pilots.shape[0]
print 'Experiment 1: Baseline: Total number of units: %s\n' % exp1_units.shape[0]

Experiment 1: Baseline: Total number of successful runs: 5
Experiment 1: Baseline: Total number of pilots: 5
Experiment 1: Baseline: Total number of units: 7



In [8]:
sws = exp1_sessions[
    ['nunit','TTC','P_LRMS_QUEUING','U_AGENT_QUEUING','U_AGENT_SCHEDULING','U_AGENT_QUEUING_EXEC','U_AGENT_EXECUTING']
        ].groupby(['nunit']).aggregate(
                {'TTC'                 : ['mean','std'],
                 'P_LRMS_QUEUING'      : ['mean','std'],
                 'U_AGENT_QUEUING'     : ['mean','std'],
                 'U_AGENT_SCHEDULING'  : ['mean','std'],
                 'U_AGENT_QUEUING_EXEC': ['mean','std'],
                 'U_AGENT_EXECUTING'   : ['mean','std']})

sws.rename({'P_LRMS_QUEUING'      :'TTQ'}, axis='columns', inplace=True)
sws.rename({'U_AGENT_QUEUING'     :'Agent queueing'}, axis='columns', inplace=True)
sws.rename({'U_AGENT_SCHEDULING'  :'Agent scheduling'}, axis='columns', inplace=True)
sws.rename({'U_AGENT_QUEUING_EXEC':'Agent queueing execution'}, axis='columns', inplace=True)
sws.rename({'U_AGENT_EXECUTING'   :'TTX'}, axis='columns', inplace=True)
sws.rename(index={i: '%s/%s' % (i, i*48) for i in sws.index.get_values()}, inplace=True)
sws

TTQ            Agent queueing           Agent queueing execution                    TTX                     TTC              Agent scheduling          
            mean        std           mean       std                     mean       std         mean       std          mean          std             mean       std
nunit                                                                                                                                                               
1/48   59.459004  42.339110       0.000788  0.000076                 0.000792  0.000037   900.610458  0.160406  1.562854e+09  5136.458185         0.003607  0.000579
2/96   41.296116   9.726954       0.001011  0.000068                 0.007377  0.000055  1800.520036  0.003263  1.562866e+09  1323.779111         0.005231  0.000490